
`Основные цели этого задания:`

- Научиться генерировать негативы.

- Научиться настраивать алгоритмы коллаборативной фильтрации.

`Задача:`

`Научиться рекомендовать пользователям фильмы на основе факта просмотра фильмов пользователями. `

`План решения:`

Для решения задачи будем использовать те же данные, которые были использованы в скринкастах:

MovieLens — источник данных.

Предобработанные для обучения данные: история проставления оценок фильмам — ratings_df_sample_2.csv.


Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать. 

- Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

- С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм

Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

Среди сгенерированных пар могут быть и дубликаты, удалите их.

Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.

Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

`Подготовьте датасет к обучению:` отделите тестовую часть от тренировочной.

`Обучите dummy-model.` Пусть она будет возвращать `случайную` вероятность принадлежности `классу 1`. Для этого можете использовать функцию `random.random` (документация). Оцените ее качество какой-то метрикой на свой вкус. Необходимо `прогнозировать именно вероятность`, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.

`Реализуйте три алгоритма коллаборативной фильтрации`: user-, item-based и алгоритм на основе матричной факторизации. 

- Оцените их качество и адекватность. 

- Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.

Опишите `вывод`, содержащий информацию о том, `какой алгоритм проявил себя лучше всего`.

----------------------

In [61]:
import pandas as pd
import numpy as np
import scipy.sparse as sp

import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, f1_score

from scipy.sparse.linalg import svds

pd.options.display.max_colwidth = 2000
pd.options.display.float_format = '{:.2f}'.format

In [2]:
ratings = pd.read_csv('ratings_df_sample_2.csv')
movies = pd.read_csv('movies.csv')

In [3]:
ratings.sort_values(by='movieId', ascending=True, inplace=True)

In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6040099 entries, 1071353 to 1099925
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 230.4 MB


- в ДФ ratings 4 фичи и 6040099 наблюдений

In [5]:
ratings.head(3)

,userId,movieId,rating,timestamp
1071353,43849,1,3.00,1017926714
5625543,6465,1,5.00,975982656
2544259,106217,1,3.50,1252286296


In [6]:
ratings.describe()

,userId,movieId,rating,timestamp
count,6040099.00,6040099.00,6040099.00,6040099.00
mean,68804.82,4822.96,3.55,1115774334.98
std,40102.24,11368.03,1.00,135843321.30
min,7.00,1.00,0.50,824835410.00
25%,34180.00,919.00,3.00,995660158.00
50%,68914.00,1876.00,4.00,1111706240.00
75%,103281.00,3448.00,4.00,1213151458.50
max,138493.00,81845.00,5.00,1427780469.00


In [7]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


- в ДФ movies 3 фичи и 62423  наблюдения

In [9]:
print(ratings.isna().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


- Пропуски в ДФ ratings отсутствуют

In [10]:
print(movies.isna().sum())

movieId    0
title      0
genres     0
dtype: int64


- Пропуски в ДФ movies отсутствуют

------
- сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

- создам новый, обобщенный ДФ

In [11]:
df = (ratings.merge(movies, on='movieId'))

df['fact_viewed_movie'] = 1

- Пропуски в ДФ ratings отсутствуют

In [69]:
display(df[:3], df.shape)

,userId,movieId,rating,timestamp,title,genres,fact_viewed_movie
0,0,0,3.00,1017926714,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
1,1,0,5.00,975982656,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
2,2,0,3.50,1252286296,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1


(6040099, 7)

Отмасштабируем идентификаторы пользователей таким образом, чтобы они начинались с 0 и заканчивались на n_users-1

Этот метод позволит находить пользовтеля по индексу матрицы (i-я строка матрицы это i-й пользователь)

In [15]:
%%time
movies_values = df['movieId'].unique()
df['movieId'] = df['movieId'].apply(lambda x: np.where(movies_values == x)[0][0])

CPU times: total: 36.5 s
Wall time: 37.1 s


In [16]:
%%time
users_values = df['userId'].unique()
df['userId'] = df['userId'].apply(lambda x: np.where(users_values == x)[0][0])

CPU times: total: 1min 11s
Wall time: 1min 12s


In [17]:
df

,userId,movieId,rating,timestamp,title,genres,fact_viewed_movie
0,0,0,3.00,1017926714,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
1,1,0,5.00,975982656,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
2,2,0,3.50,1252286296,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
3,3,0,3.00,974444867,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
4,4,0,3.50,1269167989,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1
...,...,...,...,...,...,...,...
6040094,7581,999,4.00,1312695816,"King's Speech, The (2010)",Drama,1
6040095,10965,999,4.00,1404250384,"King's Speech, The (2010)",Drama,1
6040096,3281,999,5.00,1304048880,"King's Speech, The (2010)",Drama,1
6040097,7588,999,3.00,1297197875,"King's Speech, The (2010)",Drama,1


In [18]:
df.describe()

,userId,movieId,rating,timestamp,fact_viewed_movie
count,6040099.00,6040099.00,6040099.00,6040099.00,6040099.00
mean,9449.93,485.88,3.55,1115774334.98,1.00
std,5550.58,280.26,1.00,135843321.30,0.00
min,0.00,0.00,0.50,824835410.00,1.00
25%,4672.00,250.00,3.00,995660158.00,1.00
50%,9348.00,472.00,4.00,1111706240.00,1.00
75%,14091.00,729.00,4.00,1213151458.50,1.00
max,19999.00,999.00,5.00,1427780469.00,1.00


- для обучения модели `сгенерируем` `пары`, где пользователь фильм не смотрел, они называются `"НЕГАТИВЫ"`

для этого требуется найти id всех пользователей и уникальные id всех фильмов

In [19]:
users = df.userId.unique()
movies = df.movieId.unique()

In [20]:
df.shape

(6040099, 7)

In [21]:
random_pairs_dw = []
for _ in range(df.shape[0]*2):
    user = random.choice(users)
    movie = random.choice(movies)
    random_pairs_dw.append((user, movie))

In [22]:
random_pairs_dw = pd.DataFrame(random_pairs_dw)
random_pairs_dw.columns = ['userId', 'movieId']
random_pairs_dw.head(3)

,userId,movieId
0,8995,587
1,6788,312
2,4708,181


In [23]:
random_pairs_dw.shape

(12080198, 2)

Покажем те пары в которых пользователь не смотрел фильм

In [24]:
random_pairs_w = df[['userId', 'movieId']].copy()

In [25]:
random_pairs_dw = set(tuple(x) for x in random_pairs_dw.values)
random_pairs_w = set(tuple(x) for x in random_pairs_w.values)

In [26]:
random_pairs_dw = pd.DataFrame(random_pairs_dw.difference(random_pairs_w))
random_pairs_dw.head(3)

,0,1
0,15557,614
1,15505,424
2,18177,143


In [27]:
random_pairs_dw.shape

(6330090, 2)

In [28]:
random_pairs_dw = random_pairs_dw.sample(6040099)
random_pairs_dw.columns = ['userId', 'movieId']

Объединим данные 

In [35]:
df_new = pd.concat([df, random_pairs_dw], ignore_index=True)
df_new.drop(columns=['title', 'genres'], inplace=True)
df_new

,userId,movieId,rating,timestamp,fact_viewed_movie
0,0,0,3.00,1017926714.00,1.00
1,1,0,5.00,975982656.00,1.00
2,2,0,3.50,1252286296.00,1.00
3,3,0,3.00,974444867.00,1.00
4,4,0,3.50,1269167989.00,1.00
...,...,...,...,...,...
12080193,6041,966,NaN,NaN,NaN
12080194,1395,748,NaN,NaN,NaN
12080195,11392,497,NaN,NaN,NaN
12080196,8389,717,NaN,NaN,NaN


In [36]:
df_new[['userId', 'movieId']].duplicated().sum()

0

- Заполним пропуски нулями

In [37]:
df_new = df_new.fillna(0)
df_new

,userId,movieId,rating,timestamp,fact_viewed_movie
0,0,0,3.00,1017926714.00,1.00
1,1,0,5.00,975982656.00,1.00
2,2,0,3.50,1252286296.00,1.00
3,3,0,3.00,974444867.00,1.00
4,4,0,3.50,1269167989.00,1.00
...,...,...,...,...,...
12080193,6041,966,0.00,0.00,0.00
12080194,1395,748,0.00,0.00,0.00
12080195,11392,497,0.00,0.00,0.00
12080196,8389,717,0.00,0.00,0.00


- Создадим наборы для обучения

In [38]:
train, test = train_test_split(df_new, test_size=0.05, random_state=45)

features_train = train.drop(['fact_viewed_movie'], axis = 1)
target_train = train['fact_viewed_movie']

features_test = test.drop(['fact_viewed_movie'], axis = 1)
target_test = test['fact_viewed_movie']

display('Размер выборки для тренировки', features_train.shape,target_train.shape)
display('Размер выборки для теста', features_test.shape,target_test.shape)

'Размер выборки для тренировки'

(11476188, 4)

(11476188,)

'Размер выборки для теста'

(604010, 4)

(604010,)

- Обучим простую Dummy-Model. Пусть она рэндомом возвращает вероятность принадлежности к классу 1

In [39]:
def dummy_model (features):
    prediction_P = []
    for i in range (len(features)):
        prediction_P.append(random.random())
    return prediction_P

In [45]:
pred_dummy = dummy_model(features_train)
roc_auc_dummy_model = roc_auc_score(target_train,pred_dummy)
print('метрика Roc_Auc = ', roc_auc_dummy_model)

метрика Roc_Auc =  0.4998282134444103


- `Все верно. Для простой модели метрика roc-auc и должна быть 0.5 !!`

-----------
- `Реализуем 3 алгоритма коллаборативной фильтрации: user-Based, item-Based и алгоритм на основе матричной факторизации.`

In [48]:
n_users = train['userId'].nunique()
n_movies = train['movieId'].nunique()
(n_users, n_movies)

(20000, 1000)

In [ ]:
n_users = train['userId'].nunique()
n_movies = train['movieId'].nunique()
(n_users, n_movies)

(20000, 1000)

In [ ]:
n_users = train['userId'].nunique()
n_movies = train['movieId'].nunique()
(n_users, n_movies)

(20000, 1000)

In [51]:
train_matrix = np.array(pd.pivot_table(train, values='fact_viewed_movie', index='userId', columns='movieId', fill_value=0))
train_matrix.shape

(20000, 1000)

In [52]:
train_matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

- Произведем попарное вычисление косинусного расстояния для пользователей и фильмов.

In [55]:
user_dist = cosine_distances(train_matrix)
user_dist, user_dist.shape

array([[0.        , 0.75836104, 0.65458933, ..., 0.92132353, 0.84365026,
        0.70124728],
       [0.75836104, 0.        , 0.65984208, ..., 0.72335558, 0.78238611,
        0.79704214],
       [0.65458933, 0.65984208, 0.        , ..., 0.84844223, 0.81134687,
        0.85480749],
       ...,
       [0.92132353, 0.72335558, 0.84844223, ..., 0.        , 0.66437569,
        0.56737015],
       [0.84365026, 0.78238611, 0.81134687, ..., 0.66437569, 0.        ,
        0.61564416],
       [0.70124728, 0.79704214, 0.85480749, ..., 0.56737015, 0.61564416,
        0.        ]])

(20000, 20000)

---------------------------

- `user-Based алгоритм`

для каждого user составим список из 10 пользователей похожих на него

In [56]:
top = 10
top_near_users = []
for i in range(n_users):
    n = user_dist[i].argsort()[1:top+1]
    top_near_users.append(train_matrix[n])
top_near_users = np.array(top_near_users)

In [57]:
top_near_users.shape

(20000, 10, 1000)

- создадим матрицу предсказаний, оценив популярность среди пользователей и их соседей относительно других фильмов, вычислив среднее значение а ТОП-10

In [58]:
pred_matrix_user_based = top_near_users.mean(1)
pred_matrix_user_based.shape

(20000, 1000)

- сделаем предсказания для тестовой выборки

In [59]:
test['pred_user_based'] = test.apply(lambda x: round(pred_matrix_user_based[int(x['userId']), int(x['movieId'])]), axis=1)
test.head(3)

,userId,movieId,rating,timestamp,fact_viewed_movie,pred_user_based
5594551,11859,901,3.50,1111472681.00,1.00,1
66943,14507,9,5.00,1156435569.00,1.00,1
3745537,2320,599,3.50,1304130805.00,1.00,1


In [60]:
test['pred_user_based'].unique()

array([1, 0], dtype=int64)

In [62]:
prediction = test.pred_user_based
y = test.fact_viewed_movie

roc_auc_user_based = roc_auc_score(y,prediction)
f1_user_based  = f1_score(y,prediction)

print('значение метрики roc_auc для user_based модели ', roc_auc_user_based)
print('значение метрики f1 для user_based модели ', f1_user_based)

значение метрики roc_auc для user_based модели  0.7773055761227187
значение метрики f1 для user_based модели  0.780314834595873


- модель user_based дает хорошие результаты. Выводы мы можем сделать исходя из метрик: 

значение метрики `roc_auc для user_based` модели  0.7773055761227187\
значение метрики `f1 для user_based` модели  0.780314834595873

----------------

- `Item-based алгоритм`

In [82]:
movies_dist = cosine_distances(train_matrix.T)

In [87]:
top = 10
top_near_movies = []
for i in range(n_movies):
    n = movies_dist[i].argsort()[1:top+1]
    top_near_movies.append(train_matrix.T[n])
top_near_movies = np.array(top_near_movies)
top_near_movies.shape

(1000, 10, 20000)

In [88]:
pred_matrix_item_based = top_near_movies.mean(1).T
pred_matrix_item_based.shape

(20000, 1000)

In [89]:
test['pred_item_based'] = test.apply(lambda x: round(pred_matrix_item_based[int(x['userId']), int(x['movieId'])]), axis=1)
test.head(3)

,userId,movieId,rating,timestamp,fact_viewed_movie,pred_user_based,svd_predictions,pred_item_based
5594551,11859,901,3.50,1111472681.00,1.00,1,1,1
66943,14507,9,5.00,1156435569.00,1.00,1,1,1
3745537,2320,599,3.50,1304130805.00,1.00,1,1,1


In [91]:
pred_item = test.pred_item_based

roc_auc_item_based = roc_auc_score(y,pred_item)
f1_item_based  = f1_score(y,pred_item)

print('значение метрики roc_auc для item_based модели ', roc_auc_item_based)
print('значение метрики f1 для item_based модели ', f1_item_based)

значение метрики roc_auc для item_based модели  0.7607850045058543
значение метрики f1 для item_based модели  0.7721516791289003


------

- `алгоритм на основе Матричной факторизации`

In [70]:
train_matrix.astype(float).shape

(20000, 1000)

In [74]:
u, s, vh = svds(train_matrix, k = 50)
s_diag_matrix = np.diag(s)
users = np.dot(u,s_diag_matrix)
items = vh.T
s_diag_matrix.shape, users.shape, items.shape

((50, 50), (20000, 50), (1000, 50))

In [77]:
test['svd_predictions'] = test.apply(lambda x: round(np.dot(users[int(x['userId'])], items[int(x['movieId'])])), axis=1)

In [78]:
test.head()

,userId,movieId,rating,timestamp,fact_viewed_movie,pred_user_based,svd_predictions
5594551,11859,901,3.50,1111472681.00,1.00,1,1
66943,14507,9,5.00,1156435569.00,1.00,1,1
3745537,2320,599,3.50,1304130805.00,1.00,1,1
6093189,13223,505,0.00,0.00,0.00,0,0
2262259,17667,358,3.00,1010356966.00,1.00,1,1


In [92]:
roc_auc_avd = roc_auc_score(y, test['svd_predictions'])
print('значение метрики roc_auc для матрицы факторизации ', roc_auc_avd)

roc_auc_f1 = f1_score(y, test['svd_predictions'], average='weighted')
print('значение метрик f1_score для матрицы факторизации ', roc_auc_f1)

значение метрики roc_auc для матрицы факторизации  0.762937515007499
значение метрик f1_score для матрицы факторизации  0.7551152671272212


----------

`ВЫВОДЫ:`

значение метрики `roc_auc` для:\
item_based модели -  0.76\
`user_based модели - 0.78`\
матрицы факторизации - 0.76


значение метрики `f1` для:\
item_based модели -  0.77\
`user_based модели - 0.78`\
матрицы факторизации - 0.75

- `user_based модель очень хорошо справилась с задачей на данном наборе данных`.


Существует несколько причин, почему модель, основанная на пользовательских данных (user-based модель), может хорошо справиться с задачей рекомендации:

1. Персонализация: User-based модель анализирует предпочтения и поведение конкретных пользователей. Это позволяет модели учитывать индивидуальные предпочтения и вкусы каждого пользователя, что может привести к более точным рекомендациям.

2. Социальная схожесть: User-based модель ищет схожих пользователей на основе их предпочтений и рейтингов. Если два пользователей имеют похожие предпочтения, то модель может рекомендовать элементы, которые один пользователь оценил положительно, другому пользователю с похожими предпочтениями.

3. Простота и интерпретируемость: User-based модель относительно проста в реализации и понимании. Она основана на интуитивном предположении, что пользователи с похожими предпочтениями будут иметь схожие рейтинги. Это делает модель более интерпретируемой и позволяет объяснить рекомендации пользователям.

Однако, стоит отметить, что user-based модель также имеет свои ограничения. Например, она может столкнуться с проблемой "холодного старта", когда у новых пользователей или новых элементов нет достаточного количества данных для рекомендаций. Также, при увеличении числа пользователей и элементов, вычислительная сложность модели может возрасти.